In [1]:
import pyiron_base
pyiron_base.__file__

'/Users/janssen/projects/pyiron_base/pyiron_base/__init__.py'

In [2]:
from pyiron_base import Project, job
from pyiron_base.project.delayed import draw, DelayedObject

In [3]:
def remove_server_obj(nodes_dict, edges_lst):
    server_lst = [k for k in nodes_dict.keys() if k.startswith("server_obj_")]
    for s in server_lst:
        del nodes_dict[s]
        edges_lst = [ep for ep in edges_lst if s not in ep]
    return nodes_dict, edges_lst

In [4]:
def get_nodes(connection_dict, delayed_object_updated_dict):
    return {connection_dict[k]: v._python_function if isinstance(v, DelayedObject) else v for k, v in delayed_object_updated_dict.items()}

In [5]:
def get_unique_objects(nodes_dict, edges_lst):
    delayed_object_dict = {k:v for k, v in nodes_dict.items() if isinstance(v, DelayedObject)}
    unique_lst = []
    delayed_object_updated_dict, match_dict = {}, {}
    for dobj in delayed_object_dict.keys():
        match = False
        for obj in unique_lst:
            if delayed_object_dict[dobj]._input == delayed_object_dict[obj]._input:
                delayed_object_updated_dict[obj] = delayed_object_dict[obj]
                match_dict[dobj] = obj
                match = True
                break
        if not match:
            unique_lst.append(dobj)
            delayed_object_updated_dict[dobj] = delayed_object_dict[dobj]
    delayed_object_updated_dict.update({k:v for k, v in nodes_dict.items() if not isinstance(v, DelayedObject)})
    return delayed_object_updated_dict, match_dict

In [6]:
def get_connection_dict(delayed_object_updated_dict, match_dict):
    new_obj_dict = {}
    connection_dict = {}
    lookup_dict = {}
    for i, [k, v] in enumerate(delayed_object_updated_dict.items()):
        new_obj_dict[i] = v
        connection_dict[k] = i
        lookup_dict[i] = k
    
    for k, v in match_dict.items():
        if v in connection_dict.keys():
            connection_dict[k] = connection_dict[v]

    return connection_dict, lookup_dict

In [7]:
def get_edges_dict(edges_lst, nodes_dict, connection_dict, lookup_dict):
    edges_dict_lst = []
    existing_connection_lst = []
    for ep in edges_lst:
        input_name, output_name = ep
        target = connection_dict[input_name]
        target_handle = "_".join(output_name.split("_")[:-1])
        connection_name = lookup_dict[target] + "_" + target_handle
        if connection_name not in existing_connection_lst:
            output = nodes_dict[output_name]
            if isinstance(output, DelayedObject):
                edges_dict_lst.append({
                    "target": target,
                    "targetHandle": target_handle,
                    "source": connection_dict[output_name],
                    "sourceHandle": output._output_key,
                })
            else:
                edges_dict_lst.append({
                    "target": target,
                    "targetHandle": target_handle,
                    "source": connection_dict[output_name],
                    "sourceHandle": None,
                })
            existing_connection_lst.append(connection_name)
    return edges_dict_lst

# Define Workflow 

In [8]:
pr = Project("test")

In [9]:
pr.remove_jobs(recursive=True, silently=True)

  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
@job(output_key_lst=["x", "y", "z"])
def add_x_and_y(x, y):
    z = x + y
    return {"x": x, "y": y, "z": z}

In [11]:
@job
def add_x_and_y_and_z(x, y, z):
    w = x + y + z
    return w

In [12]:
pr.remove_jobs(recursive=True, silently=True)

0it [00:00, ?it/s]

In [13]:
obj = add_x_and_y(x=1, y=2, pyiron_project=pr)

In [14]:
w = add_x_and_y_and_z(x=obj.output.x, y=obj.output.y, z=obj.output.z, pyiron_project=pr)
w.pull()

The job add_x_and_y_cd28c038ce53d240c26a6cf400f276da was saved and received the ID: 14
The job add_x_and_y_and_z_dda0dc165e1cd9391eab3794ee1cae4c was saved and received the ID: 15


6

In [15]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,14,finished,None,add_x_and_y_cd28c038ce53d240c26a6cf400f276da,/add_x_and_y_cd28c038ce53d240c26a6cf400f276da,None,/Users/janssen/notebooks/2024/2024-11-21-python-workflow-definition/test/,2024-11-21 16:09:12.885692,None,None,pyiron@mac.mpie.de#1,PythonFunctionContainerJob,0.4,None,None
1,15,finished,None,add_x_and_y_and_z_dda0dc165e1cd9391eab3794ee1cae4c,/add_x_and_y_and_z_dda0dc165e1cd9391eab3794ee1cae4c,None,/Users/janssen/notebooks/2024/2024-11-21-python-workflow-definition/test/,2024-11-21 16:09:12.996943,None,None,pyiron@mac.mpie.de#1,PythonFunctionContainerJob,0.4,None,None


In [16]:
w = add_x_and_y_and_z(x=obj.output.x, y=obj.output.y, z=obj.output.z, pyiron_project=pr)
w.pull()

6

# Convert to universal format

In [17]:
nodes_dict, edges_lst = w.get_graph()

In [18]:
nodes_dict, edges_lst = remove_server_obj(nodes_dict=nodes_dict, edges_lst=edges_lst)

In [19]:
delayed_object_updated_dict, match_dict = get_unique_objects(nodes_dict=nodes_dict, edges_lst=edges_lst)

In [20]:
connection_dict, lookup_dict = get_connection_dict(delayed_object_updated_dict=delayed_object_updated_dict, match_dict=match_dict)

In [21]:
get_nodes(connection_dict=connection_dict, delayed_object_updated_dict=delayed_object_updated_dict)

{0: <function __main__.add_x_and_y_and_z(x, y, z)>,
 1: <function __main__.add_x_and_y(x, y)>,
 2: 1,
 3: 2}

In [22]:
get_edges_dict(edges_lst=edges_lst, nodes_dict=nodes_dict, connection_dict=connection_dict, lookup_dict=lookup_dict)

[{'target': 0, 'targetHandle': 'x', 'source': 1, 'sourceHandle': 'x'},
 {'target': 1, 'targetHandle': 'x', 'source': 2, 'sourceHandle': None},
 {'target': 1, 'targetHandle': 'y', 'source': 3, 'sourceHandle': None},
 {'target': 0, 'targetHandle': 'y', 'source': 1, 'sourceHandle': 'y'},
 {'target': 0, 'targetHandle': 'z', 'source': 1, 'sourceHandle': 'z'}]